# Import

In [23]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow
Next word prediction after n_input words learned from text file
A story is automatically generated if the predicted word is fed back as input
Based on Rowel Atienza's code
'''

from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

import numpy as np
import random
import collections

import time

# Record Elapsed time

In [24]:
def time_elapsed(t):

    if t<60:
        return str(t) + " second(s)"
    elif t<(60*60):
        return str(t/60) + " minute(s)"
    else:
        return str(t/(60*60)) + " hour(s)"

# Build Dataset

In [25]:
# read word data from files
def read_data(fname):
    
    with open(fname) as f:
        content = f.readlines()
    # strip useless indent    
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content


#module for building dictionary and reverse dictionary.
def build_dataset(words):
    #Return a list of the n most common elements and their counts from the most common to the least. 
    #If n is omitted or None, most_common() returns all elements in the counter.
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        #key of the dictionary is the word and take the index of frequencies as the corresponding value
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

# Function for Building RNN Layers

In [26]:
def module_RNN(x, weights, biases, select):
    
    #weights and biases are dictionaries with a key as 'out' and content as a matrix or an array
    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (e.g. [A] [slave] [named] -> [9] [10] [37])
    x = tf.split(x,n_input,1)
    
    # basic RNN cell
    #rnn_cell = rnn.BasicRNNCell(n_hidden)

    # 1-layer LSTM with n_hidden units
    if select==1:
        rnn_cell = rnn.BasicLSTMCell(n_hidden)
    
    elif select==2:
    # 2-layer LSTM, each layer has n_hidden units. And you can wrap more layers together by doing list comprehension.
        rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 3-layer LSTM, each layer has n_hidden units. 
    # And you can wrap more layers together by doing list comprehension.
    #rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # generate prediction
    # creates a recurrent neural network specified by RNNCell cell
    outputs, states = rnn.static_rnn(rnn_cell, x,dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

# Parameters and Load Data

In [27]:
# Parameters
learning_rate = 0.001

display_step = 10



# number of words to be put into the network
n_inputs = [3,4]
training_iterss = [2000,5000,10000]
n_output = 8
selects=[1,2]
n_hiddens = [2,3,4]


# Text file containing words for training
training_file = 'simpleStory1.txt'
#training_file = 'simpleStory2.txt'
#training_file = 'androcles.txt'

training_data = read_data(training_file)
print("Loaded training data...")

dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)



Loaded training data...


In [29]:
n_input,training_iters,select,n_hidden

(3, 2000, 1, 2)

In [30]:
training_data

array(['David', 'had', 'lunch', 'with', 'Sarah', '.', 'Sarah', 'had',
       'dinner', 'with', 'Steve', '.', 'Steve', 'had', 'breakfast',
       'with', 'David', '.'], dtype='<U9')

In [31]:
dictionary

{'had': 0,
 'with': 1,
 '.': 2,
 'David': 3,
 'Sarah': 4,
 'Steve': 5,
 'lunch': 6,
 'dinner': 7,
 'breakfast': 8}

# Main Function

In [51]:
def choice(n):
    a=0
    for i in range(2):
        for j in range(3):
            for k in range(2):
                for m in range(3):
                    a+=1
                    if a==n:
                        return n_inputs[i],training_iterss[j],selects[k],n_hiddens[m]

n=1
n_input,training_iters,select,n_hidden=choice(n)


In [54]:
file1=open("Story1.txt","a")

for n in range(1,37):
    n_input,training_iters,select,n_hidden=choice(n)
    tf.reset_default_graph()

    # TensorFlow Graph input
    x = tf.placeholder("float", [None, n_input, 1])
    y = tf.placeholder("float", [None, vocab_size])


    # Logs_path
    logs_path = 'RNNs'
    writer  = tf.summary.FileWriter(logs_path)

    # RNN output node weights and biases
    weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))}
    biases  = {'out': tf.Variable(tf.random_normal([vocab_size]))}

    # get the prediction in the probability form from the output of wraped RNN layers
    pred = module_RNN(x, weights, biases,select)

    # Loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

    # Model evaluation
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initializing the variables
    init = tf.global_variables_initializer()

    start_time = time.time()

    # Launch the graph
    with tf.Session() as session:
        session.run(init)
        step = 0
        offset = random.randint(0, n_input+1)
        end_offset = n_input + 1
        acc_total = 0
        loss_total = 0

        writer.add_graph(session.graph)

        while step < training_iters:

            # Generate a mini-batch
            # Add some randomness on selection process

            if offset > (len(training_data)-end_offset):
                offset = random.randint(0, n_input+1)

            #select size of n_input data from the training set and reshape them.
            symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
            symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

            #set up the one-hot-encoding of the labels and reshape them
            symbols_out_onehot = np.zeros([vocab_size], dtype=float)
            symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
            symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

            # run optimizer,accuracy,cost and pred tensors and get the results
            _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], feed_dict={x: symbols_in_keys, y: symbols_out_onehot})

            loss_total += loss
            acc_total += acc

            #display the trend of loss and accuracy
            if (step+1) % display_step == 0:
                print("Iter = " + str(step+1) + ", average loss= " + \
                      "{:0.6f}".format(loss_total/display_step) + ", average accuracy= " + \
                      "{:0.2f}%".format(100*acc_total/display_step))
                #reset
                acc_total = 0
                loss_total = 0

                #have a brief view of the relationship of input, real value and prediction result
                symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
                symbols_out = training_data[offset + n_input]
                symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]

                print("%s - [%s] vs [%s]" % (symbols_in, symbols_out, symbols_out_pred))
            step += 1
            offset += (n_input+1)
        print("Optimization Finished!")

        t_elapsed = time.time() - start_time

        file1.write("Elapsed time: "+str(time_elapsed(t_elapsed))+"\n")

        print("Elapsed time: ", time_elapsed(t_elapsed))

        #Take a simple experiment using customized input
        if n_input==3:
            inputs=['Sarah had breakfast', 'David with Steve','Steve had lunch']
        else:
            inputs=['David had breakfast dinner', 'Sarah with David Steve', 'Sarah Steve had lunch']
            
        file1.write("The parameters are\n n_input="+str(n_input)+"\n training_iters="+str(training_iters)+"\n LSTM layer="+str(select)+"\n n_hidden="+str(n_hidden)+"\n")
        print('The paramenters are n_input=%d, training iters=%d, LSTM layer=%d and n_hidden=%d .'%(n_input,training_iters,select,n_hidden))
        for i in range(3):



    #         prompt = "insert %s words: " % n_input
    #         sentence = input(prompt)
            sentence=inputs[i]
            sentence = sentence.strip()
            words = sentence.split(' ')

            if len(words) != n_input:
                continue
            try:
                symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]

                # how many words

                for i in range(n_output):
                    keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                    onehot_pred = session.run(pred, feed_dict={x: keys})
                    onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                    sentence = "%s %s" % (sentence, reverse_dictionary[onehot_pred_index])
                    symbols_in_keys = symbols_in_keys[1:]
                    symbols_in_keys.append(onehot_pred_index)

                file1.write(sentence+"\n")
                print(sentence)
            except:
                print("cannot be found in dictionary")

        file1.write('\n \n')

file1.close()

Iter = 10, average loss= 2.068814, average accuracy= 20.00%
['breakfast', 'with', 'David'] - [.] vs [breakfast]
Iter = 20, average loss= 2.067911, average accuracy= 20.00%
['breakfast', 'with', 'David'] - [.] vs [breakfast]
Iter = 30, average loss= 2.435177, average accuracy= 0.00%
['Sarah', '.', 'Sarah'] - [had] vs [breakfast]
Iter = 40, average loss= 2.421564, average accuracy= 0.00%
['Steve', 'had', 'breakfast'] - [with] vs [breakfast]
Iter = 50, average loss= 2.367357, average accuracy= 0.00%
['had', 'dinner', 'with'] - [Steve] vs [breakfast]
Iter = 60, average loss= 2.275787, average accuracy= 10.00%
['lunch', 'with', 'Sarah'] - [.] vs [breakfast]
Iter = 70, average loss= 2.250025, average accuracy= 10.00%
['Steve', 'had', 'breakfast'] - [with] vs [breakfast]
Iter = 80, average loss= 2.447980, average accuracy= 0.00%
['Steve', '.', 'Steve'] - [had] vs [breakfast]
Iter = 90, average loss= 2.225514, average accuracy= 10.00%
['Sarah', '.', 'Sarah'] - [had] vs [breakfast]
Iter = 100, 

Iter = 470, average loss= 2.152027, average accuracy= 10.00%
['dinner', 'with', 'Steve'] - [.] vs [breakfast]
Iter = 480, average loss= 2.194593, average accuracy= 0.00%
['David', 'had', 'lunch'] - [with] vs [breakfast]
Iter = 490, average loss= 2.113434, average accuracy= 10.00%
['lunch', 'with', 'Sarah'] - [.] vs [breakfast]
Iter = 500, average loss= 2.280225, average accuracy= 0.00%
['Steve', '.', 'Steve'] - [had] vs [breakfast]
Iter = 510, average loss= 2.234646, average accuracy= 0.00%
['dinner', 'with', 'Steve'] - [.] vs [breakfast]
Iter = 520, average loss= 2.286628, average accuracy= 0.00%
['.', 'Sarah', 'had'] - [dinner] vs [breakfast]
Iter = 530, average loss= 2.150963, average accuracy= 0.00%
['lunch', 'with', 'Sarah'] - [.] vs [breakfast]
Iter = 540, average loss= 2.210463, average accuracy= 0.00%
['dinner', 'with', 'Steve'] - [.] vs [breakfast]
Iter = 550, average loss= 2.084141, average accuracy= 10.00%
['Sarah', '.', 'Sarah'] - [had] vs [breakfast]
Iter = 560, average lo

['David', 'had', 'lunch'] - [with] vs [breakfast]
Iter = 1230, average loss= 1.978444, average accuracy= 10.00%
['Steve', 'had', 'breakfast'] - [with] vs [with]
Iter = 1240, average loss= 2.064306, average accuracy= 10.00%
['Steve', '.', 'Steve'] - [had] vs [Steve]
Iter = 1250, average loss= 1.923491, average accuracy= 20.00%
['Sarah', 'had', 'dinner'] - [with] vs [Steve]
Iter = 1260, average loss= 1.924756, average accuracy= 10.00%
['had', 'lunch', 'with'] - [Sarah] vs [breakfast]
Iter = 1270, average loss= 2.062355, average accuracy= 20.00%
['.', 'Steve', 'had'] - [breakfast] vs [breakfast]
Iter = 1280, average loss= 1.873771, average accuracy= 30.00%
['Steve', 'had', 'breakfast'] - [with] vs [with]
Iter = 1290, average loss= 1.917198, average accuracy= 20.00%
['Sarah', 'had', 'dinner'] - [with] vs [Steve]
Iter = 1300, average loss= 1.959568, average accuracy= 10.00%
['breakfast', 'with', 'David'] - [.] vs [breakfast]
Iter = 1310, average loss= 1.939543, average accuracy= 30.00%
['ha

['.', 'Sarah', 'had'] - [dinner] vs [Sarah]
Optimization Finished!
Elapsed time:  10.115012168884277 second(s)
The paramenters are n_input=3, training iters=2000, LSTM layer=1 and n_hidden=2 .
Sarah had breakfast with . Sarah . . Sarah . .
David with Steve . . Sarah . . Sarah . .
Steve had lunch . . Sarah . . Sarah . .
Iter = 10, average loss= 3.191308, average accuracy= 10.00%
['Steve', '.', 'Steve'] - [had] vs [Sarah]
Iter = 20, average loss= 3.632662, average accuracy= 0.00%
['lunch', 'with', 'Sarah'] - [.] vs [Sarah]
Iter = 30, average loss= 2.667679, average accuracy= 20.00%
['had', 'breakfast', 'with'] - [David] vs [Sarah]
Iter = 40, average loss= 3.045932, average accuracy= 10.00%
['had', 'dinner', 'with'] - [Steve] vs [Sarah]
Iter = 50, average loss= 3.568327, average accuracy= 0.00%
['dinner', 'with', 'Steve'] - [.] vs [Sarah]
Iter = 60, average loss= 2.519787, average accuracy= 20.00%
['had', 'dinner', 'with'] - [Steve] vs [Sarah]
Iter = 70, average loss= 3.472231, average ac

['with', 'Steve', '.'] - [Steve] vs [Sarah]
Iter = 490, average loss= 2.372592, average accuracy= 20.00%
['had', 'lunch', 'with'] - [Sarah] vs [Sarah]
Iter = 500, average loss= 2.331128, average accuracy= 10.00%
['with', 'Steve', '.'] - [Steve] vs [Sarah]
Iter = 510, average loss= 2.498786, average accuracy= 10.00%
['lunch', 'with', 'Sarah'] - [.] vs [Sarah]
Iter = 520, average loss= 2.288081, average accuracy= 20.00%
['had', 'breakfast', 'with'] - [David] vs [Sarah]
Iter = 530, average loss= 2.433111, average accuracy= 10.00%
['Sarah', '.', 'Sarah'] - [had] vs [Sarah]
Iter = 540, average loss= 2.296354, average accuracy= 20.00%
['with', 'Sarah', '.'] - [Sarah] vs [Sarah]
Iter = 550, average loss= 2.231506, average accuracy= 20.00%
['had', 'dinner', 'with'] - [Steve] vs [Sarah]
Iter = 560, average loss= 2.408477, average accuracy= 20.00%
['had', 'lunch', 'with'] - [Sarah] vs [Sarah]
Iter = 570, average loss= 2.395047, average accuracy= 10.00%
['Steve', 'had', 'breakfast'] - [with] vs [

...

['Steve', '.', 'Steve', 'had'] - [breakfast] vs [breakfast]
Iter = 7070, average loss= 0.758807, average accuracy= 70.00%
['with', 'Steve', '.', 'Steve'] - [had] vs [had]
Iter = 7080, average loss= 0.560295, average accuracy= 80.00%
['lunch', 'with', 'Sarah', '.'] - [Sarah] vs [Sarah]
Iter = 7090, average loss= 0.745333, average accuracy= 70.00%
['had', 'lunch', 'with', 'Sarah'] - [.] vs [.]
Iter = 7100, average loss= 0.849933, average accuracy= 70.00%
['had', 'breakfast', 'with', 'David'] - [.] vs [.]
Iter = 7110, average loss= 0.957872, average accuracy= 70.00%
['Steve', 'had', 'breakfast', 'with'] - [David] vs [Steve]
Iter = 7120, average loss= 0.537238, average accuracy= 80.00%
['Sarah', 'had', 'dinner', 'with'] - [Steve] vs [Steve]
Iter = 7130, average loss= 0.621199, average accuracy= 80.00%
['Sarah', '.', 'Sarah', 'had'] - [dinner] vs [breakfast]
Iter = 7140, average loss= 0.557244, average accuracy= 80.00%
['had', 'breakfast', 'with', 'David'] - [.] vs [.]
Iter = 7150, average 

['had', 'dinner', 'with', 'Steve'] - [.] vs [.]
Iter = 7790, average loss= 0.856420, average accuracy= 60.00%
['had', 'lunch', 'with', 'Sarah'] - [.] vs [.]
Iter = 7800, average loss= 0.464242, average accuracy= 90.00%
['Sarah', 'had', 'dinner', 'with'] - [Steve] vs [Steve]
Iter = 7810, average loss= 0.939767, average accuracy= 60.00%
['Sarah', '.', 'Sarah', 'had'] - [dinner] vs [breakfast]
Iter = 7820, average loss= 0.560029, average accuracy= 80.00%
['.', 'Steve', 'had', 'breakfast'] - [with] vs [with]
Iter = 7830, average loss= 0.934194, average accuracy= 70.00%
['with', 'Steve', '.', 'Steve'] - [had] vs [had]
Iter = 7840, average loss= 0.664990, average accuracy= 70.00%
['lunch', 'with', 'Sarah', '.'] - [Sarah] vs [Sarah]
Iter = 7850, average loss= 0.600774, average accuracy= 70.00%
['Steve', 'had', 'breakfast', 'with'] - [David] vs [Steve]
Iter = 7860, average loss= 0.914210, average accuracy= 70.00%
['with', 'Steve', '.', 'Steve'] - [had] vs [had]
Iter = 7870, average loss= 0.480

['dinner', 'with', 'Steve', '.'] - [Steve] vs [Sarah]
Iter = 8510, average loss= 0.288515, average accuracy= 100.00%
['Sarah', 'had', 'dinner', 'with'] - [Steve] vs [Steve]
Iter = 8520, average loss= 0.800980, average accuracy= 70.00%
['Steve', '.', 'Steve', 'had'] - [breakfast] vs [breakfast]
Iter = 8530, average loss= 0.357985, average accuracy= 90.00%
['Steve', '.', 'Steve', 'had'] - [breakfast] vs [breakfast]
Iter = 8540, average loss= 0.471866, average accuracy= 80.00%
['.', 'Sarah', 'had', 'dinner'] - [with] vs [with]
Iter = 8550, average loss= 0.505622, average accuracy= 70.00%
['Steve', '.', 'Steve', 'had'] - [breakfast] vs [breakfast]
Iter = 8560, average loss= 0.624669, average accuracy= 80.00%
['with', 'Steve', '.', 'Steve'] - [had] vs [had]
Iter = 8570, average loss= 0.428130, average accuracy= 80.00%
['Steve', 'had', 'breakfast', 'with'] - [David] vs [Steve]
Iter = 8580, average loss= 0.721309, average accuracy= 80.00%
['had', 'dinner', 'with', 'Steve'] - [.] vs [.]
Iter =

['Sarah', '.', 'Sarah', 'had'] - [dinner] vs [breakfast]
Iter = 9230, average loss= 0.569046, average accuracy= 80.00%
['Sarah', '.', 'Sarah', 'had'] - [dinner] vs [breakfast]
Iter = 9240, average loss= 0.416694, average accuracy= 90.00%
['David', 'had', 'lunch', 'with'] - [Sarah] vs [Sarah]
Iter = 9250, average loss= 0.867057, average accuracy= 60.00%
['dinner', 'with', 'Steve', '.'] - [Steve] vs [Sarah]
Iter = 9260, average loss= 0.401378, average accuracy= 90.00%
['Steve', 'had', 'breakfast', 'with'] - [David] vs [Steve]
Iter = 9270, average loss= 0.397889, average accuracy= 90.00%
['lunch', 'with', 'Sarah', '.'] - [Sarah] vs [Sarah]
Iter = 9280, average loss= 0.570172, average accuracy= 70.00%
['had', 'lunch', 'with', 'Sarah'] - [.] vs [.]
Iter = 9290, average loss= 0.359985, average accuracy= 100.00%
['.', 'Sarah', 'had', 'dinner'] - [with] vs [with]
Iter = 9300, average loss= 0.762972, average accuracy= 60.00%
['Sarah', '.', 'Sarah', 'had'] - [dinner] vs [breakfast]
Iter = 9310, 

['Steve', 'had', 'breakfast', 'with'] - [David] vs [Steve]
Iter = 9950, average loss= 0.598380, average accuracy= 80.00%
['.', 'Sarah', 'had', 'dinner'] - [with] vs [with]
Iter = 9960, average loss= 0.297610, average accuracy= 100.00%
['David', 'had', 'lunch', 'with'] - [Sarah] vs [Sarah]
Iter = 9970, average loss= 0.244685, average accuracy= 100.00%
['.', 'Sarah', 'had', 'dinner'] - [with] vs [with]
Iter = 9980, average loss= 0.311558, average accuracy= 90.00%
['had', 'breakfast', 'with', 'David'] - [.] vs [.]
Iter = 9990, average loss= 0.292123, average accuracy= 100.00%
['David', 'had', 'lunch', 'with'] - [Sarah] vs [Sarah]
Iter = 10000, average loss= 0.477325, average accuracy= 80.00%
['.', 'Sarah', 'had', 'dinner'] - [with] vs [with]
Optimization Finished!
Elapsed time:  1.9626646836598713 minute(s)
The paramenters are n_input=4, training iters=10000, LSTM layer=2 and n_hidden=4 .
David had breakfast dinner with Steve with Steve with Sarah with Sarah
Sarah with David Steve with ha

In [47]:
file1.write("Elapsed time:"+str(5))
file1.close()